In [21]:
from dotenv import load_dotenv
from typing import Any, Dict, List
from operator import itemgetter

from langchain.chat_models import init_chat_model
from langchain_core.runnables import RunnableLambda
from typing_extensions import TypedDict
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langgraph.prebuilt import create_react_agent
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

from pydantic import BaseModel, Field

In [4]:
load_dotenv()

True

In [5]:
class Args(TypedDict):
    a: int
    b: List[int]

def f(x: Args) -> str:
    return str(x["a"] * max(x["b"]))

runnable = RunnableLambda(f)
as_tool = runnable.as_tool(
    name="My tool",
    description="Explanation of when to use tool.",
)

/var/folders/fl/jdrswfjx50v2304lpb3s4wxr0000gn/T/ipykernel_82849/2701302436.py:9: LangChainBetaWarning: This API is in beta and may change in the future.
  as_tool = runnable.as_tool(


In [6]:
print(as_tool.description)
as_tool.args_schema.schema()

Explanation of when to use tool.


/var/folders/fl/jdrswfjx50v2304lpb3s4wxr0000gn/T/ipykernel_82849/3738499149.py:2: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  as_tool.args_schema.schema()


{'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}},
 'required': ['a', 'b'],
 'title': 'My tool',
 'type': 'object'}

In [7]:
as_tool.invoke({"a":3, "b":[1,2]})

'6'

In [9]:
def g(x: Dict[str, Any]) -> str:
    return str(x["a"] * max(x["b"]))


runnable = RunnableLambda(g)
as_tool = runnable.as_tool(
    name="My tool",
    description="Explanation of when to use tool.",
    arg_types={"a": int, "b": List[int]},
)

In [11]:
class GSchema(BaseModel):
    """Apply a function to an integer and list of integers."""

    a: int = Field(..., description="Integer")
    b: List[int] = Field(..., description="List of ints")


runnable = RunnableLambda(g)
as_tool = runnable.as_tool(GSchema)

In [12]:
def f(x: str) -> str:
    return x + "a"


def g(x: str) -> str:
    return x + "z"


runnable = RunnableLambda(f) | g
as_tool = runnable.as_tool()

In [13]:
as_tool.invoke("b")

'baz'

In [15]:
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [17]:
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
    ),
]

vectorstore = InMemoryVectorStore.from_documents(
    documents, embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

In [19]:
tools = [
    retriever.as_tool(
        name="pet_info_retriever",
        description="Get information about pets.",
    )
]
agent = create_react_agent(llm, tools)

In [20]:
for chunk in agent.stream({"messages": [("human", "What are dogs known for?")]}):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Dogs are known for a variety of traits and characteristics, including:\n\n1. **Loyalty**: Dogs are often called "man’s best friend" due to their unwavering loyalty to their owners and families.\n\n2. **Companionship**: They provide emotional support and companionship, making them popular pets in households worldwide.\n\n3. **Intelligence**: Many dog breeds are known for their intelligence and ability to learn commands, solve problems, and perform tricks.\n\n4. **Sense of Smell**: Dogs have an exceptional sense of smell, which makes them useful in roles such as search and rescue, hunting, and detection of drugs or explosives.\n\n5. **Diverse Breeds**: There are hundreds of dog breeds, each with unique physical characteristics, temperaments, and skill sets, making them suitable for various roles, from working dogs to lap dogs.\n\n6. **Communication**: Dogs use vocalizations (barking, growling) and body language (tail wagging, ear positioning) to

In [22]:
system_prompt = """
You are an assistant for question-answering tasks.
Use the below context to answer the question. If
you don't know the answer, say you don't know.
Use three sentences maximum and keep the answer
concise.

Answer in the style of {answer_style}.

Question: {question}

Context: {context}
"""

prompt = ChatPromptTemplate.from_messages([("system", system_prompt)])

rag_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "answer_style": itemgetter("answer_style"),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
rag_chain.input_schema.schema()

/var/folders/fl/jdrswfjx50v2304lpb3s4wxr0000gn/T/ipykernel_82849/2875943523.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  rag_chain.input_schema.schema()


{'properties': {'question': {'title': 'Question'},
  'answer_style': {'title': 'Answer Style'}},
 'required': ['question', 'answer_style'],
 'title': 'RunnableParallel<context,question,answer_style>Input',
 'type': 'object'}

In [24]:
rag_tool = rag_chain.as_tool(
    name="pet_expert",
    description="Get information about pets.",
)

In [25]:
agent = create_react_agent(llm, [rag_tool])

for chunk in agent.stream(
    {"messages": [("human", "What would a pirate say dogs are known for?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5SgU5yyEJg2HXWP1hv0ujlnC', 'function': {'arguments': '{"question":"What are dogs known for?","answer_style":"pirate"}', 'name': 'pet_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 58, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_27322b4e16', 'id': 'chatcmpl-BFR8INBt8C4DOUkiTnJKQi0eKcAN9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5d154942-2fba-49ad-a41c-d9f569261288-0', tool_calls=[{'name': 'pet_expert', 'args': {'question': 'What are dogs known for?', 'answer_style': 'pirate'}, 'id': 'call_5SgU5yyEJg2HXWP1hv0ujlnC', 'type': 'tool_call'}], usage_metadata={'i